In [2]:
import os
api_key = os.getenv("DEEPSEEK_API_KEY")
base_url = os.getenv("DEEPSEEK_BASE_URL")

In [17]:
# 准备数据

items = []
mfd_file_path = "./mfd.md"
with open(mfd_file_path, "r") as file:
    file_text:str = file.read()
    # 找到所有法规
    handle_items = file_text.split("\n**")
    # 过滤不是法规的条目
    handle_items = [s for s in handle_items if '**' in s]
    # 去掉**
    handle_items = [s.replace("**", "") for s in handle_items]
    items += handle_items

In [18]:
# 准备LLM和embeding模型
from openai import OpenAI

deepseek_client = OpenAI(api_key=api_key,base_url=base_url)

In [19]:
from pymilvus import model as milvus_model

embedding_model = milvus_model.DefaultEmbeddingFunction()

/root/miniconda3/envs/deepseek-quickstart/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [20]:
test_embedding = embedding_model.encode_queries(["This is a test"])[0]
embedding_dim = len(test_embedding)
print(embedding_dim)
print(test_embedding[:10])

768
[-0.04836059  0.07163021 -0.01130063 -0.03789341 -0.03320651 -0.01318453
 -0.03041721 -0.02269495 -0.02317858 -0.00426026]


In [23]:
# 将数据加载到Milvus

from pymilvus import MilvusClient

milvus_client = MilvusClient(uri="./milvus_mfd.db")

collection_name = "mfd_collection"

In [24]:
if milvus_client.has_collection(collection_name):
    milvus_client.drop_collection(collection_name)

In [25]:
milvus_client.create_collection(
    collection_name=collection_name,
    dimension=embedding_dim,
    metric_type="IP",
    consistency_level="Strong"
)

In [29]:
# 插入数据

from tqdm import tqdm

data = []

doc_embeddings = embedding_model.encode_documents(items)

for i, item in enumerate(tqdm(items, desc="Creating embeddings")):
    data.append({"id": i, "vector": doc_embeddings[i], "text": item})

milvus_client.insert(collection_name=collection_name, data=data)

Creating embeddings: 100%|██████████| 387/387 [00:00<00:00, 479384.42it/s]


{'insert_count': 387, 'ids': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 

In [34]:
# 检索查询数据

question_1 = "所有权人对自己的不动产或者动产，有什么权利？"
question_2 = "抵押权所担保的债权有哪些？"

In [35]:
search_res = milvus_client.search(
    collection_name=collection_name,
    data=embedding_model.encode_queries([question_1, question_2]),
    limit=3,
    search_params={"metric_type": "IP", "params": {}},
    output_fields=["text"]
)

In [37]:
print(search_res)

data: [[{'id': 19, 'distance': 0.8118640184402466, 'entity': {'text': '第二百二十三条 动产物权的设立和转让，自交付时发生效力，但是法律另有规定的除外。\n'}}, {'id': 10, 'distance': 0.8118640184402466, 'entity': {'text': '第二百一十四条 不动产物权的设立、变更、转让和消灭，依照法律规定应当登记的，自记载于不动产登记簿时发生效力。\n'}}, {'id': 4, 'distance': 0.8118640184402466, 'entity': {'text': '第二百零八条 不动产权利的设立、变更、转让和消灭，应当依照法律规定登记。动产物权的设立和转让，应当依照法律规定交付。\n'}}], [{'id': 111, 'distance': 0.7870465517044067, 'entity': {'text': '第三百一十四条 按份共有人对共有的不动产或者动产按照其份额享有所有权。\n'}}, {'id': 74, 'distance': 0.7870465517044067, 'entity': {'text': '第二百七十七条 业主不得以放弃权利为由不履行义务。\n'}}, {'id': 1, 'distance': 0.7870465517044067, 'entity': {'text': '第二百零五条 本编调整因物的归属和利用产生的民事关系。\n'}}]]


In [43]:
import json

retrieved_lines_with_distances = [
    (res["entity"]["text"], res["distance"]) for res in search_res[0]
]
print(json.dumps(retrieved_lines_with_distances, indent=4, ensure_ascii=False))

[
    [
        "第二百二十三条 动产物权的设立和转让，自交付时发生效力，但是法律另有规定的除外。\n",
        0.8118640184402466
    ],
    [
        "第二百一十四条 不动产物权的设立、变更、转让和消灭，依照法律规定应当登记的，自记载于不动产登记簿时发生效力。\n",
        0.8118640184402466
    ],
    [
        "第二百零八条 不动产权利的设立、变更、转让和消灭，应当依照法律规定登记。动产物权的设立和转让，应当依照法律规定交付。\n",
        0.8118640184402466
    ]
]


In [45]:
context1 = "\n".join([res["entity"]["text"] for res in search_res[0]])
print(context1)

第二百二十三条 动产物权的设立和转让，自交付时发生效力，但是法律另有规定的除外。

第二百一十四条 不动产物权的设立、变更、转让和消灭，依照法律规定应当登记的，自记载于不动产登记簿时发生效力。

第二百零八条 不动产权利的设立、变更、转让和消灭，应当依照法律规定登记。动产物权的设立和转让，应当依照法律规定交付。



In [46]:
context2 = "\n".join([res["entity"]["text"] for res in search_res[1]])
print(context2)

第三百一十四条 按份共有人对共有的不动产或者动产按照其份额享有所有权。

第二百七十七条 业主不得以放弃权利为由不履行义务。

第二百零五条 本编调整因物的归属和利用产生的民事关系。



In [47]:
print(question_1)
print(question_2)

所有权人对自己的不动产或者动产，有什么权利？
抵押权所担保的债权有哪些？


In [48]:
SYSTEM_PROMPT = """
Human: 你是一个 AI 助手。你能够从提供的上下文段落片段中找到问题的答案。
"""
USER_PROMPT = """
请使用以下用 <context> 标签括起来的信息片段来回答用 <question> 标签括起来的问题。
最后用<remark>和</remark>来标注对 <context> 内容的准确性评价，以及添加你自己不参考 <context> 信息片段的回答。

<context>
{context}
</context>
<question>
{question}
</question>
<remark>
</remark>
"""

In [50]:
USER_PROMPT_1 = USER_PROMPT.format(context=context1, question=question_1)
print(USER_PROMPT_1)


请使用以下用 <context> 标签括起来的信息片段来回答用 <question> 标签括起来的问题。
最后用<remark>和</remark>来标注对 <context> 内容的准确性评价，以及添加你自己不参考 <context> 信息片段的回答。

<context>
第二百二十三条 动产物权的设立和转让，自交付时发生效力，但是法律另有规定的除外。

第二百一十四条 不动产物权的设立、变更、转让和消灭，依照法律规定应当登记的，自记载于不动产登记簿时发生效力。

第二百零八条 不动产权利的设立、变更、转让和消灭，应当依照法律规定登记。动产物权的设立和转让，应当依照法律规定交付。

</context>
<question>
所有权人对自己的不动产或者动产，有什么权利？
</question>
<translated>
</translated>



In [51]:
USER_PROMPT_2 = USER_PROMPT.format(context=context2, question=question_2)
print(USER_PROMPT_2)


请使用以下用 <context> 标签括起来的信息片段来回答用 <question> 标签括起来的问题。
最后用<remark>和</remark>来标注对 <context> 内容的准确性评价，以及添加你自己不参考 <context> 信息片段的回答。

<context>
第三百一十四条 按份共有人对共有的不动产或者动产按照其份额享有所有权。

第二百七十七条 业主不得以放弃权利为由不履行义务。

第二百零五条 本编调整因物的归属和利用产生的民事关系。

</context>
<question>
抵押权所担保的债权有哪些？
</question>
<translated>
</translated>



In [52]:
response = deepseek_client.chat.completions.create(
    model="deepseek-chat",
    messages=[
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": USER_PROMPT_1},
    ],
)
print(response.choices[0].message.content)

<answer>
根据提供的上下文信息，虽然未直接说明所有权人的具体权利，但可以从物权变动规则反推：
1. 对于动产（第二百二十三条）：所有权人通过"交付"实现物权的设立和转让
2. 对于不动产（第二百一十四条、第二百零八条）：所有权人通过"登记"实现物权的设立、变更、转让和消灭

由此可推知所有权人至少享有：处分权（转让/消灭物权）、设立他物权等权利。

<remark>
• 准确性评价：上下文确实完整引用了中国《民法典》物权编相关条款，但仅涉及物权变动规则，未直接规定所有权权能
• 典型的所有权权能还应包括：占有、使用、收益等基本权能，这些在《民法典》第二百四十条有明确规定（未在提供的上下文中出现）
</remark>

补充说明（不依赖上下文）：
根据中国《民法典》第二百四十条，所有权人对自己的不动产或动产依法享有：
1. 占有权
2. 使用权
3. 收益权
4. 处分权
这是所有权的四项基本权能，上下文中的条款实际是处分权的具体实现方式。
</answer>


In [53]:
response = deepseek_client.chat.completions.create(
    model="deepseek-chat",
    messages=[
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": USER_PROMPT_2},
    ],
)
print(response.choices[0].message.content)

<remark>
提供的<context>内容并未包含与"抵押权所担保的债权"直接相关的法律条文信息。上下文主要涉及的是按份共有、业主权利义务以及物权编的调整范围，这些内容与抵押权担保的债权类型没有直接关联。
</remark>

在不参考<context>信息的情况下回答：
根据中国《民法典》相关规定，抵押权所担保的债权主要包括：
1. 主债权（如贷款本金）
2. 利息（包括法定利息和约定利息）
3. 违约金
4. 损害赔偿金
5. 实现抵押权的费用（如评估费、拍卖费等）
6. 保管抵押财产的费用（如需要）

具体规定可参见《民法典》第四百一十条等相关条款。
